![CAT_DOG](img/banniere.png)

In [ ]:
# Import des dépendances python nécessaire 

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

import mlflow
from mlflow.models import infer_signature
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

np.set_printoptions(linewidth=np.inf)
np.set_printoptions(edgeitems=30) 


# Exploration des données 

Ici nous allons simplement affciher une radio pour vérifier qu'on arrive à la lire. 

In [ ]:
# On charge une image depuis un chemin local, l’image est chargé en 1 canal (valeur de pixel de 0 à 255)
img = cv2.imread("data/train/NORMAL/IM-0115-0001.jpeg", cv2.IMREAD_GRAYSCALE)

# Observer l'image sous forme de matrice
#print(img)

# Visualisation de l'image
plt.imshow(img, cmap="gray")
plt.axis("off")
plt.show()

# Inspection des métadonnées de l'image
print(f"Shape (dimensions)    : {img.shape}")
print(f"Type des valeurs      : {img.dtype}")


Ici nous avons une radiographie qui est affich donc tout est bon 

# Préparation des données 

In [ ]:
def get_train_test(base_path: str, img_size=(224,224)):
    X = []  # liste pour stocker les images
    y = []  # liste pour stocker les étiquettes correspondantes

    # Dictionnaire pour mapper les étiquettes de chaînes à des entiers
    label_map = {'NORMAL': 0, 'PNEUMONIA': 1}

    # On parcourt les sous-dossiers du répertoire
    for label in sorted(os.listdir(base_path)):
        # On ignore les fichiers qui ne sont pas des dossiers de classes
        if label not in label_map:
            continue
        label_path = os.path.join(base_path, label)

        # On parcourt chaque image du dossier
        for file_name in os.listdir(label_path):
            file_path = os.path.join(label_path, file_name)
            # Lecture de l'image en niveaux de gris
            img = cv2.imread(file_path)
            if img is None:
                continue  # image illisible, on passe
            
            # Redimensionnement de l'image
            img = cv2.resize(img, img_size)  # Redimensionner en 380x380 pixels
            X.append(img)           # on ajoute l'image à la liste
            y.append(label_map[label])  # on ajoute le label numérique

    # Conversion des listes en tableaux NumPy
    X = np.array(X)
    y = np.array(y)

    # Reshape pour les convolutions
    X = X.reshape(X.shape[0], img_size[0], img_size[1], 3)  # (n_samples, height, width, channels)
    return X, y

X_train, y_train = get_train_test("data/train")
X_test, y_test = get_train_test("data/test")
X_valid, y_valid= get_train_test("data/val")

In [ ]:
# Vérification des données préparées
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(X_train[i], cmap="grey")
    plt.axis("off")

plt.show()

les images sont corects, on continue 

In [ ]:
# Test images aléatoire 

# Supposons que X_train est votre ensemble de données d'images
# Nombre d'images à afficher
num_images = 10

# Sélectionner des indices aléatoires
random_indices = np.random.choice(len(X_train), num_images, replace=False)

# Vérification des données préparées
plt.figure(figsize=(15, 5))  # Ajustez la taille de la figure si nécessaire
for i, idx in enumerate(random_indices):
    plt.subplot(1, num_images, i + 1)
    plt.imshow(X_train[idx], cmap="grey")
    plt.axis("off")

plt.show()

# Preprocessing

In [ ]:
# # Les images sont au format (380, 380). On les convertit en float pour normaliser ensuite
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_valid = X_valid.astype("float32")

# # Normalisation : on divise les valeurs de pixels par 255 pour les ramener entre 0 et 1
X_train /= 255.0
X_test /= 255.0
X_valid /= 255.0

# X_train = np.repeat(X_train, 3, axis=-1)  # Convertir en 3 canaux
# X_test = np.repeat(X_test, 3, axis=-1)    # Convertir en 3 canaux
# X_valid = np.repeat(X_valid, 3, axis=-1)    # Convertir en 3 canaux

# Création modele 

In [ ]:
# definition taille image pour model 

input_shape= (224,224,3)
inputs= Input(shape=input_shape)
# modele de base pour EfficientNetB4 
base = EfficientNetB0(include_top=False, input_tensor=inputs, weights='imagenet')
base.trainable = False  # Freeze

In [ ]:
# Ajoute featues efficient 
features = base.output
features = GlobalAveragePooling2D()(features)
features = Dropout(0.4)(features)
features = Dense(128, activation='relu')(features)
features = Dropout(0.3)(features)

outputs= Dense(1, activation='sigmoid')(features)

In [ ]:
# création model 

model = Model(inputs=inputs, outputs=outputs)

#compilation 
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entraînement
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=5, batch_size=16)
#model.summary()

In [ ]:

base.trainable = True
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(
    X_train,y_train,
    validation_data=(X_valid, y_valid),
    batch_size=32,
    epochs=10,
    shuffle=True
          )